# 0. Quick overview of Python, Jupyter, and some of their features

We will go through the basics features of Python and Jupyter in order to plot, create datasets, use dataframes, and do fits.

If you have never used Jupyter: Jupyter notebooks are a very cool and practical way of doing "simple analysis" and teaching. It allows an interactive framework to use Python, and at the same time allows the insertion fo comments and descriptions like the one you are reading now.

This kind of cell is known as a MARKDOWN cell. Apart from this kind of cell, there is the more useful one for analysis: the CODE cell, like the following in which we import the necessary libraries of Python that we will use today.

In [ ]:
import matplotlib                     # as the name says is a library for plotting in Python
import matplotlib.pyplot as plt
import numpy as np                    # library of numerical python functions
import pandas as pd                   # library for data handling and data formats
import scipy.stats as ss              # library of statistical methods
from scipy.optimize import curve_fit  # function for fitting
import uproot3                         # library to read .root files "whithout ROOT"

## 0.1 Creating histograms and filling them

In this section we give a quick look to histograms and binning.
There are two main methods you can use in python to deal with a histogram (i.e. a binned series):
 * `plt.hist(series, bins)`: you use this method in case you want to plot directly the histogram. This methods returns `n`, the values of the histogram bins, `bins`, the edges of the bins and `patches`, silent list of individual patches used to create the histogram or list of such list if multiple input datasets (patches are almost never used).
 * `np.hisogram(series, bins)`: compute the histogram of a set of data. This method returns `hist`, the values of the histogram and `bin_edges`, the bin edges `(length(hist)+1)`.

Let's create some dummy data.

We start by creating an array of $10^5$ random numbers, generated according to a `crystallball distribution`. Then we create two histograms using both `plt.hist()` and `np.histogram()`.

In [ ]:
# generate the values distributed as a gaussian
beta, m = 0.4, 5 
cb_vals = ss.crystalball.rvs(beta, m, size=10000)

In [ ]:
binning = np.linspace(-20, 5, 200)
n, bins, patches = plt.hist(cb_vals, binning, density=True)

Calling `plt.hist` automatically plots the histogram of the series we are providing to this method. Also note that it is preferable to assing `plt.hist()` to a variable, otherwise you also get the printout of all the contents in each bin. For example, try to run:

In [ ]:
#plt.hist(cb_vals, binning)

Therefore, if we do not directly need to see the histogram, it is better to use `np.hisogram` (especially if you do not want the plots to appear and make your notebook longer and longer).

In [ ]:
hist, bin_edges = np.histogram(cb_vals, bins,  density=True)

It is worth mentioning that the bins edges returned by the two methods have length larger than the values of the histogram. Therefore, we need to remove the overflow bin when we want to plot `bins` vs `n` (or `bin_edges` vs `hist`):

In [ ]:
plt.figure(figsize=(8,4))
t = plt.hist(cb_vals, binning,  density=True)
plt.plot(bins[:-1], n, 'ko', label = 'points from plt.hist()')
plt.plot(bin_edges[:-1], hist, 'r*', label = 'points from np.histogram()')
plt.legend()

Last but not least, the bins edges above do not correspond to the bins centers. This is why in the plot above, the points are not aligned with the histogram. Let's define the bin centers and then use these points to fit the above distribution with a gaussian:

In [ ]:
bins = (bins[:-1] + bins[1:])/2
bin_edges = (bin_edges[:-1] + bin_edges[1:])/2

In [ ]:
plt.figure(figsize=(8,4))
plt.title('Aligned bin edges')
t = plt.hist(cb_vals, binning,  density=True)
plt.plot(bins, n, 'ko', label = 'points from plt.hist()')
plt.plot(bin_edges, hist, 'r*', label = 'points from np.histogram()')
plt.legend()

## 0.2 Fitting histograms 

Having generated a distribution we can fit it with a user-defined function, using the `curve_fit` method with a user defined function. Let's start by defining the `gaussian` function:

In [ ]:
def gaussian(x, A, sigma, mu):
    return A/(sigma * np.sqrt(2 * np.pi))*np.exp( - (x - mu)**2 / (2 * sigma**2) )

Now let's fit the distributuin with a gaussian. 

`curve_fit` expects 3 arguments:
 * The fit function, in our case `gaussian`;
 * The `x` and `y` arrays of data;

For more options (such as setting initial guess for the free parameters, including uncertainties to the fit and boundaries on the fit free parameters) you can have a look at the [`curve_fit` documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html).

In [ ]:
popt, pcov = curve_fit(gaussian, bins, n)

The fit results are:
 * `popt`: Optimal values for the parameters so that the sum of the squared residuals of `f(xdata, *popt) - ydata` is minimized;
 * `pcov`: The estimated covariance of popt. The diagonals provide the variance of the parameter estimate. To compute one standard deviation errors on the parameters use `perr = np.sqrt(np.diag(pcov))`.

In [ ]:
plt.figure(figsize=(8, 4))
plt.title('Fit of the distribution')
plt.plot(bins, n, 'ko', label = 'points from plt.hist()')
plt.plot(bin_edges, hist, 'r*', label = 'points from np.histogram()')
plt.plot(binning, gaussian(binning, *popt), '-', label = 'Gaussian fit')
plt.legend()

As we would expect the `gussian` does not fit well the daat. Therefore we we have to use the `crystalball.fit` fitting. As input it takes:
* the data to be fitted (`cb_vals`)
* the starting guess of the parameters to fit: in our case we start guessing the normalization with `scale=1`
Complete documentation can be found [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.crystalball.html)

In [ ]:
beta_fit, m_fit, offset_fit, norm_fit = ss.crystalball.fit(cb_vals, scale=1)
print("MLE beta =", beta_fit, "; MLE m =", m_fit, "; MLE offset =", offset_fit, "; MLE norm =", norm_fit)

The output of the fitting is the Maximum Likelihood Estimate (MLE) of the parameters of the `crystallball` distribution. Now we can plot it to visually confront the result of teh fit with the original distribution we created.

In [ ]:
plt.figure(figsize=(8, 4))
plt.title('Fit of the distribution')
plt.plot(bins, n, 'ko', label = 'points from plt.hist()')
plt.plot(bin_edges, hist, 'r*', label = 'points from np.histogram()')
plt.plot(binning, gaussian(binning, *popt), '-', label = 'Gaussian fit')
rv = ss.crystalball(beta_fit, m_fit, offset_fit, norm_fit)
plt.plot(binning, rv.pdf(binning), '-', label = 'Crystalball fit')
plt.legend()

## 0.3 Dataset handling

First of all, let's import the dataset we are going to use.
Normally HEP data come in the `.root` format, which was born to be directly usable with the [`ROOT` framework](https://root.cern.ch/). We are going to open it into a `pandas` data frame, using [`uproot`](https://github.com/scikit-hep/uproot) to import the ROOT `TTree`.
We also have to define a `key`, which is the name of the `TTree` we want to open.

In [ ]:
tree = uproot3.open('/data_CMS/cms/motta/HHLegacy_SKIMS/SKIMS2018/TREX/SKIM_GGHH_NLO_cHHH1_xs.root')['HTauTauTree']
df = tree.pandas.df(["dau1_pt", "dau1_eta", "dau1_phi", "dau1_flav"])

In [ ]:
df.head()

### 0.2.1 Selecting events is Dataframes

Now that you have in mind the structure of the data frame, we can get some pandas basics knowledge. Let's say we want to look at all the entries in our `df` where the `dau1_pt` is larger than 50GeV. In pandas this can be quickly done with a `query` on the `df`:

In [ ]:
## we ask for head() because 
## df[sel] is still a df
sel = df['dau1_pt'] > 50
df[sel].head() # or df.query('rechit_energy > 20.')

Here we have learnt how to apply a boolean selection to a data frame. The `sel` object is a pandas Series made of `True` and `False` entries according to whether the selection (`df['rechit_energy'] > 20`) is satisfied or not:

In [ ]:
sel.head()

We can also combine multiple selections and create a new df starting from the original one, but with a selection applied on data. Something like this:

In [ ]:
## We define some selections
sel_energy = df['dau1_pt'] > 50
sel_eta = df['dau1_eta'] < 2.3

## We combine the selection together.
## Note that python uses the single & (or | ).
sel_tot = sel_energy & sel_eta

## We create a new df with only the events
## we are interested on.
df_cut = df[sel_tot]

Let's now check the difference in length between these two data frames:

In [ ]:
len(df)

In [ ]:
len(df_cut)

### 0.2.1 Grouping

We can even group items, perform calculations on some aggregated property, then plot values, just to get an idea of larger scale trends. This property of grouping the elements of the data frame together can turn out to be very useful. In `pandas` you can `groupby()` a data frame by items.

In [ ]:
gb = df.groupby('dau1_flav')

In [ ]:
## gb is an object from which we can get properties on aggregrated items of df
gb

In [ ]:
## Let's say we want to check the min/max value of a variable per event. 
## Here an example for the reconstructed energy:

gb.agg({'dau1_pt':'min'}).head()

In [ ]:
gb.agg({'dau1_pt':'max'}).head()